In [31]:
import requests
from pyquery import PyQuery as pq
import re
import os
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
from time import sleep

## Scrap Chinese Insurance Data

### Wrap 2020-2021 Data From China Banking and Insurance Regulatory Commission

#### Wrap Function

In [2]:
#Get page 1-3
def get_web(page):
    r = requests.get('http://www.cbirc.gov.cn/cn/static/data/DocInfo/SelectDocByItemIdAndChild/data_itemId=954,pageIndex={page},pageSize=18.json'.format(page=page))
    return r

In [3]:
#Get page 4-6
def get_web2(page):
    cookies = {
        'yfx_c_g_u_id_10006849': '_ck21121511293510955416158987436',
        'yfx_f_l_v_t_10006849': 'f_t_1639538975091__r_t_1639538975091__v_t_1639546463631__r_c_0',
    }

    headers = {
        'Proxy-Connection': 'keep-alive',
        'Accept': '*/*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'http://www.cbirc.gov.cn/cn/view/pages/ItemList.html?itemPId=953&itemId=954&itemUrl=ItemListRightList.html&itemName=%E7%BB%9F%E8%AE%A1%E4%BF%A1%E6%81%AF',
        'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8',
    }

    params = (
        ('itemId', '954'),
        ('pageSize', '18'),
        ('pageIndex', '{page}'.format(page=page)),
    )

    r = requests.get('http://www.cbirc.gov.cn/cbircweb/DocInfo/SelectDocByItemIdAndChild', headers=headers, params=params, cookies=cookies, verify=False)
    return r

In [4]:
def get_seperate_page(docId):
    url = 'http://www.cbirc.gov.cn/cn/view/pages/ItemDetail.html?docId={docId}&itemId=954&generaltype=0'.format(docId = docId)
    #doc = pq(url)
    return url

#### Get DocId Dict

##### Get dict for 2021 total

In [11]:
pattern1 = re.compile('.*保险业经营情况表')
def parse_page(pattern,json,dic):
    if json:
        items = json.get('data').get('rows')
        for item in items:
            if re.match(pattern,item.get('docSubtitle')):
                dic[item.get('docSubtitle')] = item.get('docId')
    #    print(dic)

dic = {}
for page in range(1,4):
    r = get_web(page)
    parse_page(pattern1,r.json(),dic)
print(dic)

{'2021年10月保险业经营情况表': 1020664, '2021年9月保险业经营情况表': 1014403, '2021年8月保险业经营情况表': 1009929, '2021年7月保险业经营情况表': 1003572, '2021年6月保险业经营情况表': 1000291, '2021年5月保险业经营情况表': 992619, '2021年4月保险业经营情况表': 985887, '2021年3月保险业经营情况表': 979346, '2021年2月保险业经营情况表': 973010, '2021年1月保险业经营情况表': 969471}


In [12]:
dic2 = {}
for page in range(4,7):
    r = get_web2(page)
    parse_page(pattern1,r.json(),dic2)
dic2

{'2020年12月保险业经营情况表': 963080,
 '2020年11月保险业经营情况表': 952787,
 '2020年10月保险业经营情况表': 945039,
 '2020年9月保险业经营情况表': 937163,
 '2020年8月保险业经营情况表': 931640,
 '2020年7月保险业经营情况表': 925152,
 '2020年6月保险业经营情况表': 918275,
 '2020年5月保险业经营情况表': 911957,
 '2020年4月保险业经营情况表': 906375,
 '2020年3月保险业经营情况表': 900170,
 '2020年2月保险业经营情况表': 896051,
 '2020年1月保险业经营情况表': 894619}

In [139]:
dic5 = {}
for page in range(9,14):
    r = get_web2(page)
    parse_page(pattern1,r.json(),dic5)
dic5

{'2019年1-4月保险业经营情况表': 367741,
 '2019年1-3月保险业经营情况表': 367742,
 '2019年1-2月保险业经营情况表': 367743,
 '2019年1月保险业经营情况表': 367744,
 '2018年保险业经营情况表': 367745,
 '2018年1-11月保险业经营情况表': 367746,
 '2018年1-10月保险业经营情况表': 367747,
 '2018年1-9月保险业经营情况表': 367748,
 '2018年1-8月保险业经营情况表': 367749,
 '2018年1-7月保险业经营情况表': 367750,
 '2018年1-6月保险业经营情况表': 367751,
 '2018年1-5月保险业经营情况表': 367571,
 '2018年1-4月保险业经营情况表': 367570,
 '2018年1-3月保险业经营情况表': 367572,
 '2018年1-2月保险业经营情况表': 367573,
 '2018年1月保险业经营情况表': 367574,
 '2017年保险业经营情况表': 367575}

##### Get dict for 2021 Province

In [142]:
pattern2 = re.compile('.*全国.*保费.*')

dic3 = {}
for page in range(1,4):
    r = get_web(page)
    parse_page(pattern2,r.json(),dic3)
print(dic3)

{'2021年10月全国各地区原保险保费收入情况表': 1020667, '2021年9月全国各地区原保险保费收入情况表': 1014417, '2021年8月全国各地区原保险保费收入情况表': 1009932, '2021年7月全国各地区原保险保费收入情况表': 1003582, '2021年6月全国各地区原保险保费收入情况表': 1000302, '2021年5月全国各地区原保险保费收入情况表': 992622, '2021年4月全国各地区原保险保费收入情况表': 985925, '2021年3月全国各地区原保险保费收入情况表': 979356, '2021年2月全国各地区原保险保费收入情况表': 973023, '2021年1月全国各地区原保险保费收入情况表': 969478}


In [145]:
dic4 = {}
for page in range(4,7):
    r = get_web2(page)
    parse_page(pattern2,r.json(),dic4)
print(dic4)

{'2020年12月全国各地区原保险保费收入情况表': 963083, '2020年11月全国各地区原保险保费收入情况表': 952796, '2020年10月全国各地区原保险保费收入情况表': 945042, '2020年9月全国各地区原保险保费收入情况表': 937174, '2020年8月全国各地区原保险保费收入情况表': 931649, '2020年7月全国各地区原保险保费收入情况表': 925165, '2020年6月全国各地区原保险保费收入情况表': 918283, '2020年5月全国各地区原保险保费收入情况表': 911963, '2020年4月全国各地区原保险保费收入情况表': 906381, '2020年3月全国各地区原保险保费收入情况表': 900235, '2020年2月全国各地区原保险保费收入情况表': 896063, '2020年1月全国各地区原保险保费收入情况表': 894625}


In [144]:
dic6 = {}
for page in range(9,27):
    r = get_web2(page)
    parse_page(pattern2,r.json(),dic6)
print(dic6)

{'2019年1-3月全国各地区原保险保费收入情况表': 371582, '2019年1-2月全国各地区原保险保费收入情况表': 371581, '2019年1月全国各地区原保险保费收入情况表': 371580, '2018年全国各地区原保险保费收入情况表': 371579, '2018年1-11月全国各地区原保险保费收入情况表': 371578, '2018年1-10月全国各地区原保险保费收入情况表': 371577, '2018年1-9月全国各地区原保险保费收入情况表': 371576, '2018年1-8月全国各地区原保险保费收入情况表': 371575, '2018年1-7月全国各地区原保险保费收入情况表': 371574, '2018年1-6月全国各地区原保险保费收入情况表': 371573, '2018年1-5月全国各地区原保险保费收入情况表': 371572, '2018年1-4月全国各地区原保险保费收入情况表': 371571, '2018年1-3月全国各地区原保险保费收入情况表': 371570, '2018年1-2月全国各地区原保险保费收入情况表': 371569, '2018年1月全国各地区原保险保费收入情况表': 371568, '2017年全国各地区原保险保费收入情况表': 371567, '2017年1-11月全国各地区原保险保费收入情况表': 371566, '2017年1-10月全国各地区原保险保费收入情况表': 371565, '2017年1-9月全国各地区原保险保费收入情况表': 371564, '2017年1-8月全国各地区原保险保费收入情况表': 371563, '2017年1-7月全国各地区原保险保费收入情况表': 371562, '2017年1-6月全国各地区原保险保费收入情况表': 371561, '2017年1-5月全国各地区原保险保费收入情况表': 371560, '2017年1-4月全国各地区原保险保费收入情况表': 371559, '2017年1-3月全国各地区原保险保费收入情况表': 371558, '2017年1-2月全国各地区原保险保费收入情况表': 371557, '2017年1月全国各地区原保险保费收入情况表': 371556, '2016年全国各地区原保险保费收入情况表': 371555, '2016

In [22]:
def get_file_last(path):
    dir_list = os.listdir(path)
    dir_list = sorted(dir_list,key=lambda x: os.path.getmtime(os.path.join(path, x)))
    return dir_list[-1]
get_file_last("/mnt/d/data_science/Insurance_report/data/")

'raw_province'

#### Download Data Function

##### Download all-year data


In [21]:
#Driver Setting
Timeout = 10
raw_data_path = "/mnt/d/data_science/Insurance_report/data/raw/"
fp = webdriver.FirefoxProfile()
fp.set_preference("browser.download.folderList", 2)        
fp.set_preference("browser.download.dir", raw_data_path)
fp.set_preference("browser.download.manager.showWhenStarting",False)
fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel;application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

def get_one_file(driver,key,val):
    count = len(os.listdir(raw_data_path))
    driver.get(get_seperate_page(val))
    driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[2]/div/div[5]/ul/li/a').click()
    if len(os.listdir(raw_data_path)) == count + 1:
        file = raw_data_path + get_file_last(raw_data_path)
        os.rename(file, raw_data_path + '{key}.xlsx'.format(key=key))
        print("Success")




##### Download province data

In [33]:
raw_data_path_province = "/mnt/d/data_science/Insurance_report/data/raw_province/"
fp2 = webdriver.FirefoxProfile()
fp2.set_preference("browser.download.folderList", 2)        
fp2.set_preference("browser.download.dir", raw_data_path_province)
fp2.set_preference("browser.download.manager.showWhenStarting",False)
fp2.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel;application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

button = '/html/body/div[2]/div[3]/div[2]/div[2]/div/div[5]/ul/li/a'
def get_one_file_province(driver,key,val):
    count = len(os.listdir(raw_data_path_province))
    driver.get(get_seperate_page(val))
    wait = WebDriverWait(driver,10)
    input = wait.until(EC.presence_of_element_located((By.XPATH,button)))
    input.find_element_by_xpath(button).click()
    if len(os.listdir(raw_data_path_province)) == count + 1:
        file = raw_data_path_province + get_file_last(raw_data_path_province)
        os.rename(file, raw_data_path_province + '{key}.xlsx'.format(key=key))
        print("Success")



##### Get all-Year data

In [ ]:
driver = webdriver.Firefox(firefox_profile=fp)

for key,val in dic.items():    
    try:
        print("The page is about {key}\n".format(key=key))
        get_one_file(driver,key,val)
        sleep(3)
    except:
        print(key,'\tFailed')
        get_one_file(driver,key,val)
        #continue

###### Test

In [15]:
driver = webdriver.Firefox(firefox_profile=fp)
driver.get(get_seperate_page((dic['2021年5月保险业经营情况表'])))
driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[2]/div/div[5]/ul/li/a').click()

In [ ]:
driver = webdriver.Firefox(firefox_profile=fp)
for key,val in dic2.items():    
    try:
        print("The page is about {key}\n".format(key=key))
        get_one_file(driver,key,val)
        sleep(3)
    except:
        print(key,'\tFailed')
        get_one_file(driver,key,val)
        #continue

##### Get province data

In [19]:
tmp ={**dic3,**dic4}

In [ ]:
driver = webdriver.Firefox(firefox_profile=fp2)
for key,val in tmp.items():    
    if f'{key}.xlsx' not in os.listdir(raw_data_path_province):
        try:
            print("The page is about {key}\n".format(key=key))
            get_one_file_province(driver,key,val)
            #break
            sleep(3)
        
        except:
            print(key,'\tFailed')
            get_one_file(driver,key,val)
            continue
    else:
        print(key + 'already existed')

### Data Processing

In [41]:
import pandas as pd
import numpy as np


#### Processing All-year data

In [38]:
def file_path(path):
    return sorted(os.listdir(raw_data_path)[:-1],key=lambda x: os.path.getmtime(os.path.join(raw_data_path, x)))
file = file_path(raw_data_path)

In [11]:
processed_path = '/mnt/d/data_science/Insurance_report/data/processed'

In [152]:
def clean_column(columns_list):
    """
    这个代码负责将表格的列名称清洗干净（就是将一些二级标题、三级标题都转换成完整的名称：{一级标题}-{二级标题}-{三级标题})
    如果标题是一级标题 -> 一级标题
    如果标题是二级标题 -> {一级标题}-{二级标题}
    如果标题是三级标题 -> {一级标题}-{二级标题}-{三级标题}

    这么一做，每一个表的列名称就不会混淆

    """

    column_data = pd.DataFrame({'column_name':[i.replace(" ", '') for i in columns_list]})
    # 做一级填充
    column_data['class_1'] = column_data['column_name'].apply(lambda x: len(re.findall(pattern='[0-9]+、', string=x)) !=1)#[len(re.findall(pattern='[0-9]+、', string=temp_column)) !=1 for temp_column in columns_list]
    column_data['class_1_name'] = column_data['column_name']
    column_data.loc[column_data['class_1']== False, 'class_1_name'] = np.nan
    column_data['class_1_name'] = column_data['class_1_name'].ffill()
    # column_data.loc[column_data['column_name'] != column_data['class_1_name'],'class_1_name'] = \
    #     column_data.loc[column_data['column_name'] != column_data['class_1_name'],:].apply(lambda x: f"{x['class_1_name']}__{x['column_name']}", axis=1)
    column_data.loc[column_data['class_1'] == False,'class_1_name'] = \
        column_data.loc[column_data['class_1'] == False,:].apply(lambda x: f"{x['class_1_name']}__{x['column_name']}", axis=1)
    # 做二级填充

    column_data['class_2'] = column_data['column_name'].apply(lambda x: len(re.findall(pattern='（[0-9]+）', string=x))!=1)

    column_data['class_2_name'] = column_data['class_1_name']
    column_data.loc[column_data['class_2'] == False, 'class_2_name'] = np.nan
    column_data['class_2_name'] = column_data['class_2_name'].ffill()

    column_data.loc[column_data['class_2'] == False, 'class_2_name'] = column_data.loc[column_data['class_2'] == False, :].apply(lambda x: f"{x['class_2_name']}__{x['column_name']}", axis=1)

    return column_data['class_2_name'].tolist()

In [62]:
def get_one_data(i):
    try:
        data = pd.read_excel(raw_data_path + i,engine = 'openpyxl',skiprows=2).drop('Unnamed: 0',axis = 1).dropna()
    except:
        data = pd.read_excel(raw_data_path + i,engine = 'xlrd',skiprows=2).drop('Unnamed: 0',axis = 1).dropna()
    data = data.set_index('项目').T.reset_index(drop=True)
    columns_list = data.columns.tolist()
    data.columns = clean_column(columns_list)
    data['info'] = '{i}'.format(i = i[:-6])
    return data
#data

In [60]:
dd = pd.DataFrame()
for i in file:
    try:
        tmp = get_one_data(i)
        dd = dd.append(tmp)
    except:
        print(i)

2021年6月保险业经营情况表.xlsx


In [ ]:
data = pd.read_excel(raw_data_path + '2021年6月保险业经营情况表.xlsx',engine = 'openpyxl',skiprows=2).drop(['Unnamed: 0','Unnamed: 3'],axis = 1).dropna()
data = data.set_index('项目').T.reset_index(drop=True)
#data
columns_list = data.columns.tolist()
data.columns = clean_column(columns_list)
data['info'] = '2021年6月保险业经营情况'
data
dd = dd.append(data)

In [52]:
import xlrd
#xlrd.open_workbook(raw_data_path + '2021年5月保险业经营情况表.xlsx')

In [73]:
dd.to_csv('../data/industry_new.csv',index=False)

#### Processing province data

In [ ]:
province_file = file_path(raw_data_path_province)
province_file

In [77]:
def get_one_data_province(i):
    try:
        data = pd.read_excel('../data/raw_province/' + i,engine = 'openpyxl',skiprows=4)
    except:
        data = pd.read_excel('../data/raw_province/' + i,engine = 'xlrd',skiprows=4)
    data = data[data.columns.drop(list(data.filter(regex='Unname')))].dropna()
    #data = data.set_index('项目').T.reset_index(drop=True)
    #columns_list = data.columns.tolist()
    #data.columns = clean_column(columns_list)
    data['info'] = '{i}'.format(i = i[:-6])
    return data

In [ ]:
get_one_data_province(os.listdir('../data/raw_province')[0])

In [80]:
dd_pro = pd.DataFrame()
for i in os.listdir('../data/raw_province'):
    try:
        tmp = get_one_data_province(i)
        dd_pro = dd_pro.append(tmp)
    except:
        print(i)
dd_pro

,地区,合计,财产保险,寿险,意外险,健康险,info
0,全国合计,39608.32,10047.86,21394.57,1003.80,7162.08,2020年10月全国各地区原保险保费收入情况
1,集团、总公司本级,57.22,50.76,0.02,2.70,3.74,2020年10月全国各地区原保险保费收入情况
2,北 京,1950.61,371.68,1120.89,58.07,399.97,2020年10月全国各地区原保险保费收入情况
3,天 津,587.32,139.67,338.10,17.64,91.92,2020年10月全国各地区原保险保费收入情况
4,河 北,1848.08,505.42,986.82,34.61,321.23,2020年10月全国各地区原保险保费收入情况
...,...,...,...,...,...,...,...
33,青 海,86.85,33.48,36.27,2.72,14.39,2021年9月全国各地区原保险保费收入情况
34,宁 夏,174.28,50.54,86.97,5.65,31.12,2021年9月全国各地区原保险保费收入情况
35,新 疆,572.72,186.97,261.52,14.22,110.00,2021年9月全国各地区原保险保费收入情况
36,内蒙古,530.66,159.82,258.55,12.38,99.91,2021年9月全国各地区原保险保费收入情况


In [81]:
dd_pro.to_csv('../data/new_data_by_province.csv',index = False)

In [72]:
xlrd.open_workbook('../data/raw_province/' + os.listdir('../data/raw_province')[0])

### Using selenium to scrape web table

#### Part 1. Scrap 18-19 All-Year Data

In [234]:
#raw_data_path_province = "/mnt/d/data_science/Insurance_report/data/raw_province/"
fp3 = webdriver.FirefoxProfile()
#fp3.set_preference("browser.download.folderList", 2)        
#fp3.set_preference("browser.download.dir", raw_data_path_province)
fp3.set_preference("browser.download.manager.showWhenStarting",False)
#fp2.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel;application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

def get_page_table(key,val):
    driver.get(f'http://www.cbirc.gov.cn/cn/view/pages/ItemDetail.html?docId={val}&itemId=954&generaltype=0')
    wait = WebDriverWait(driver,10)
    input = wait.until(EC.presence_of_all_elements_located((By.XPATH,'/html/body')))
    doc = pq(driver.page_source)
    items = doc('tr').items()
    ls = []
    for item in items:
        ls.append(item('p')('span').text())
    ls = [x for x in ls if '单位' not in x and x != '']
    if '项目' not in ls[0]:
        ls.insert(0,'项目  本年累计/截至当期')
    tmp_d = pd.Series(ls).astype(str).str.split(r'\s+(?=\d{2,}|本.*)',expand = True)
    tmp_d.columns = tmp_d.iloc[0]
    tmp_d = tmp_d[1:]
    tmp_d = tmp_d.set_index('项目').T.reset_index(drop=True)
    columns_list = tmp_d.columns.tolist()
    tmp_d.columns = clean_column(columns_list)   
    tmp_d['info'] = f'{key[:-1]}'
    return tmp_d


In [239]:
d_before = pd.DataFrame()
driver = webdriver.Firefox(firefox_profile=fp3)
for key,val in dic5.items():
    try:
        tmp = get_page_table(key,val)
        d_before = d_before.append(tmp)
        print(key,'\tsuccess')
        sleep(3)
    except:
        print(key,'\tfailed')
        tmp = get_page_table(key,val)
        d_before = d_before.append(tmp)
        sleep(3)

2018年1-7月保险业经营情况表 	failed
2018年1-7月保险业经营情况表
2018年1-4月保险业经营情况表 	failed
2018年1-4月保险业经营情况表
2018年1-3月保险业经营情况表 	failed
2018年1-3月保险业经营情况表


In [ ]:
d_before = d_before.reset_index(drop=True)
d_before.to_csv('../data/industry_old.csv',index=False)

In [ ]:
#driver = webdriver.Firefox(firefox_profile=fp3)
get_page_table('x',367744)